In [1]:
%matplotlib inline
import pandas as pd
import csv
import io
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class CSVCleaner:
    def __init__(self, csvfile):
        self.filename = csvfile
        self.cleaned = None
    
        # Read raw csv data
        self.read_raw_csv()
    
    def read_raw_csv(self):
        ''' Reading raw input from a csv file.

        Reads a csv file and returns a list of all lines.
        '''
        with io.open(self.filename, newline=None, encoding='utf-8') as f:
            data = f.read().splitlines()
            self.header = data[0].split(',')
            self.raw_data = [tuple(line.split(',')) for line in data[1:]]
        
        self.cleaned = self.raw_data
    
    def remove_duplicates(self):
        ''' Remove all duplicate lines in the data.
        '''
        cleaned = list(dict.fromkeys(self.cleaned))
        self.num_duplicates = len(self.cleaned) - len(cleaned)
        self.cleaned = cleaned
        
        return self
    
    def remove_corrupt(self):
        ''' Remove all corrupt lines in the data.
        
        Corrupt lines are defined for us as those that will
        not fit in a dataframe - that is, those that have a different
        column count as our header.
        '''
        cleaned = []
        self.num_corrupt = 0
        
        col_count = len(self.header)
        
        for row in self.cleaned:
            if len(row) == col_count:
                cleaned.append(row)
            else:
                self.num_corrupt += 1
                
        self.cleaned = cleaned
        return self
    
    def dataframe(self):
        ''' Return a pandas dataframe of the cleaned data.
        '''
        return pd.DataFrame.from_records(self.cleaned, columns=self.header)
    
    def __repr__(self):
        return "CSVCleaner(file='%s')" % (self.filename)

In [3]:
csvfile = 'dirty_sample_small.csv'

dup_corr_cleaner = CSVCleaner(csvfile).remove_duplicates().remove_corrupt()
corr_dup_cleaner = CSVCleaner(csvfile).remove_corrupt().remove_duplicates()

In [4]:
def get_stats(cleaner, title):
    print(title)
    print('Total entries: ', len(cleaner.raw_data))
    print('Cleaned entries: ', len(cleaner.cleaned))
    print('Number of duplicates removed:' , cleaner.num_duplicates)
    print('Number of corrupt entries removed: ', cleaner.num_corrupt)

In [5]:
get_stats(dup_corr_cleaner, 'Duplicates removed first, then corrupt:\n')

Duplicates removed first, then corrupt:

Total entries:  661486
Cleaned entries:  49981
Number of duplicates removed: 604264
Number of corrupt entries removed:  7241


In [6]:
get_stats(corr_dup_cleaner, 'Corrupt removed first, then duplicates:\n')

Corrupt removed first, then duplicates:

Total entries:  661486
Cleaned entries:  49981
Number of duplicates removed: 595797
Number of corrupt entries removed:  15708


In [7]:
data = dup_corr_cleaner.dataframe()

In [8]:
pd.set_option('display.max_columns', None)

In [9]:
data.head()

,course_id,user_id,registered,viewed,explored,certified,completed,ip,cc_by_ip,countryLabel,continent,city,region,subdivision,postalCode,un_major_region,un_economic_group,un_developing_nation,un_special_region,latitude,longitude,LoE,YoB,gender,grade,passing_grade,start_time,first_event,last_event,nevents,ndays_act,nplay_video,nchapters,nforum_posts,nforum_votes,nforum_endorsed,nforum_threads,nforum_comments,nforum_pinned,roles,nprogcheck,nproblem_check,nforum_events,mode,is_active,cert_created_date,cert_modified_date,cert_status
0,HarvardX/PH525.1x/1T2018,7940,False,,False,False,81.108.107.58,GB,United Kingdom,Europe,Middlesbrough,MDB,Middlesbrough,,Northern Europe,Developed regions,,,54.5728,-1.1628,,,,,0.7,2018-04-10 08:30:28,2018-04-10 08:30:28.055918,2018-04-10 08:30:38.600946,6,2,0,,,,,,,,Student,0,0,0,audit,1,,,,
1,HarvardX/PH525.1x/1T2018,21193,True,True,False,False,103.108.88.2,,,,,,,,,,,,,,b,1993,m,,0.7,2018-02-06 13:45:52,2018-02-06 00:00:00,2018-05-05 18:24:23.645179,785,11,95,5,,,,,,,Student,0,142,0,audit,0,,,,
2,HarvardX/PH525.1x/1T2018,27938,True,True,False,False,179.214.111.130,BR,Brazil,Americas,Brasília,DF,Federal,71920,South America,Developing_Nations,,Latin America and the Caribbean,-15.7833,-47.9167,b,1992,m,,0.7,2018-01-30 18:10:51,2018-01-30 18:10:50.819162,2018-05-27 01:02:43.640466,829,37,51,3,,,,,,,Student,0,207,0,audit,0,,,,
3,HarvardX/PH525.1x/1T2018,28454,True,False,False,False,177.18.230.216,BR,Brazil,Americas,Guarulhos,SP,Sao Paulo,,South America,Developing_Nations,,Latin America and the Caribbean,-23.4507,-46.5262,m,1980,m,,0.7,2018-04-24 23:35:15,2018-04-24 23:35:15.449395,2018-04-24 23:35:41.878589,9,1,0,1,,,,,,,Student,0,0,0,audit,1,,,,
4,HarvardX/PH525.1x/1T2018,43100,False,,False,False,213.249.56.36,GR,Greece,Europe,Athens,I,Attica,,Southern Europe,Developed regions,,,37.9833,23.7333,,,,,0.7,2018-01-16 15:48:29,2018-01-16 15:48:29.309103,2018-01-16 15:48:29.309103,1,1,0,,,,,,,,Student,0,0,0,audit,1,,,,


In [10]:
data.completed.unique()

array(['81.108.107.58', '103.108.88.2', '179.214.111.130', ...,
       '162.231.122.2', '94.54.53.235', '71.41.229.179'], dtype=object)

Data seems to be broken as we can clearly see that beginning with the completed column, each of the columns are shifted over by one. This would indicate that the five boolean columns ("registered", "viewed", ... "completed") are missing one column worth of data. However, we cannot know for sure how to repair these columns, as in the case of column shifting, the problem is that we have no basis for establishing which data columns belong to which header in what position. The only fix we can do is restore each of the columns afterward, and not trust any of the 5 boolean columns. If we were to be perfectly proper we would seriously consider deleting these columns straight up, but in this case we simply shift each by one, and eliminate the unknown completed column.

In [11]:
# We've manually identified the problem column
def pset2_fix(df):
    idx = 7
    cols = np.array(df.columns)
    cols[idx-1:-1]=cols[idx:]

    # Replace columns
    df.columns=cols
    return df

In [12]:
data = pset2_fix(data)

In [13]:
data.head()

,course_id,user_id,registered,viewed,explored,certified,ip,cc_by_ip,countryLabel,continent,city,region,subdivision,postalCode,un_major_region,un_economic_group,un_developing_nation,un_special_region,latitude,longitude,LoE,YoB,gender,grade,passing_grade,start_time,first_event,last_event,nevents,ndays_act,nplay_video,nchapters,nforum_posts,nforum_votes,nforum_endorsed,nforum_threads,nforum_comments,nforum_pinned,roles,nprogcheck,nproblem_check,nforum_events,mode,is_active,cert_created_date,cert_modified_date,cert_status,cert_status
0,HarvardX/PH525.1x/1T2018,7940,False,,False,False,81.108.107.58,GB,United Kingdom,Europe,Middlesbrough,MDB,Middlesbrough,,Northern Europe,Developed regions,,,54.5728,-1.1628,,,,,0.7,2018-04-10 08:30:28,2018-04-10 08:30:28.055918,2018-04-10 08:30:38.600946,6,2,0,,,,,,,,Student,0,0,0,audit,1,,,,
1,HarvardX/PH525.1x/1T2018,21193,True,True,False,False,103.108.88.2,,,,,,,,,,,,,,b,1993,m,,0.7,2018-02-06 13:45:52,2018-02-06 00:00:00,2018-05-05 18:24:23.645179,785,11,95,5,,,,,,,Student,0,142,0,audit,0,,,,
2,HarvardX/PH525.1x/1T2018,27938,True,True,False,False,179.214.111.130,BR,Brazil,Americas,Brasília,DF,Federal,71920,South America,Developing_Nations,,Latin America and the Caribbean,-15.7833,-47.9167,b,1992,m,,0.7,2018-01-30 18:10:51,2018-01-30 18:10:50.819162,2018-05-27 01:02:43.640466,829,37,51,3,,,,,,,Student,0,207,0,audit,0,,,,
3,HarvardX/PH525.1x/1T2018,28454,True,False,False,False,177.18.230.216,BR,Brazil,Americas,Guarulhos,SP,Sao Paulo,,South America,Developing_Nations,,Latin America and the Caribbean,-23.4507,-46.5262,m,1980,m,,0.7,2018-04-24 23:35:15,2018-04-24 23:35:15.449395,2018-04-24 23:35:41.878589,9,1,0,1,,,,,,,Student,0,0,0,audit,1,,,,
4,HarvardX/PH525.1x/1T2018,43100,False,,False,False,213.249.56.36,GR,Greece,Europe,Athens,I,Attica,,Southern Europe,Developed regions,,,37.9833,23.7333,,,,,0.7,2018-01-16 15:48:29,2018-01-16 15:48:29.309103,2018-01-16 15:48:29.309103,1,1,0,,,,,,,,Student,0,0,0,audit,1,,,,


In [15]:
# Check distribution of gender
data.gender.value_counts()

m       24196
f       17327
         8244
o         212
null        2
Name: gender, dtype: int64

In [18]:
# Check distribution of developed region
data.un_major_region.value_counts()

Northern America             16266
                              5614
Southern Asia                 4320
South America                 3844
Northern Europe               2655
Eastern Asia                  2610
Western Europe                2533
Southern Europe               2181
South-Eastern Asia            1942
Eastern Europe                1765
Western Asia                  1425
Central America               1163
Australia and New Zealand      947
Northern Africa                736
Western Africa                 694
Eastern Africa                 552
Caribbean                      305
Southern Africa                256
Central Asia                    89
Middle Africa                   54
Melanesia                       21
Micronesia                       5
Polynesia                        4
Name: un_major_region, dtype: int64